In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/movie-review-sentiment-analysis-kernels-only/test.tsv
/kaggle/input/movie-review-sentiment-analysis-kernels-only/train.tsv
/kaggle/input/movie-review-sentiment-analysis-kernels-only/sampleSubmission.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from scipy.sparse import hstack, vstack
import csv
import re
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import gensim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.ensemble import RandomForestRegressor
import nltk
import string
# !pip install contractions
# import contractions
# nltk.download('stopwords')
# nltk.download('wordnet')

In [3]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z ]')
STOPWORDS = set(stopwords.words('english'))
stemmer = nltk.stem.WordNetLemmatizer()

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text
#     text = " ".join([contractions.fix(t) for t in text.split(' ')])
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, '', text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([word.strip(string.punctuation) for word in text.split(' ')])
    text = ' '.join([t for t in text.split(" ") if len(t) > 2])
    text = ' '.join([stemmer.lemmatize(x) for x in text.split() if x not in STOPWORDS])# delete stopwords from text
    return text
    
    

In [4]:
df = pd.read_csv("/kaggle/input/movie-review-sentiment-analysis-kernels-only/train.tsv", sep="\t")
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


The sentiment labels are:

0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive

In [5]:
print(df.iloc[0]["Phrase"])
print(df["Sentiment"].value_counts())

A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .
2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64


In [6]:
# sample run of function
text_prepare(df.iloc[0]["Phrase"])

'series escapade demonstrating adage good goose also good gander occasionally amuses none amount much story'

In [7]:
df["Phrase"] = df["Phrase"].apply(text_prepare)

In [8]:
# tfidf = TfidfVectorizer(ngram_range=(1,2))
# features = tfidf.fit_transform(np.array(train_x)) #fit transform to learn vocubulary and transform in matrix

trainx = df["Phrase"]
trainy = df["Sentiment"]

tfidf = TfidfVectorizer(min_df=5, max_df=0.5, ngram_range=(1,3))
features = tfidf.fit_transform(np.array(trainx)) #fit transform to learn vocubulary and transform in matrix


regressor = OneVsRestClassifier(LogisticRegression())

regressor.fit(features, np.array(trainy))
# regressor.fit(np.array(question2vec_result), np.array(train_y))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [9]:
df1 = pd.read_csv("/kaggle/input/movie-review-sentiment-analysis-kernels-only/test.tsv", sep="\t")
df1.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [10]:
df1["Phrase"] = df1["Phrase"].apply(text_prepare)
testx = df1["Phrase"]

features3 = tfidf.transform(np.array(testx))


x3 = regressor.predict(features3)

x4 = pd.DataFrame({'PhraseId':np.array(df1["PhraseId"]), 'Sentiment':x3})
x4["Sentiment"] = x4["Sentiment"].apply(lambda x:int(round(x)))
x4.to_csv("output.csv", index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 